In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [2]:
# Read in CSV file
regional_df = pd.read_csv("Table_CSVs/regional.csv")
regional_df

,DOEID,REGIONC,DIVISION,state_postal,BA_climate,TOTALBTU,TOTALDOL
0,100001,WEST,Mountain South,NM,Mixed-Dry,144647.71,2656.89
1,100002,SOUTH,West South Central,AR,Mixed-Humid,28034.61,975.00
2,100003,WEST,Mountain South,NM,Mixed-Dry,30749.71,522.65
3,100004,SOUTH,South Atlantic,SC,Mixed-Humid,86765.19,2061.77
4,100005,NORTHEAST,Middle Atlantic,NJ,Mixed-Humid,59126.93,1463.04
...,...,...,...,...,...,...,...
18491,118492,SOUTH,South Atlantic,MD,Mixed-Humid,49930.49,1098.51
18492,118493,NORTHEAST,New England,ME,Very-Cold,222186.04,3613.44
18493,118494,SOUTH,West South Central,TX,Hot-Humid,51593.72,1428.31
18494,118495,SOUTH,South Atlantic,SC,Hot-Humid,63555.21,2224.94


In [3]:
# Create DataFrame with regional information
regional_df = regional_df.drop(columns='DOEID')
regional_df.head()

,REGIONC,DIVISION,state_postal,BA_climate,TOTALBTU,TOTALDOL
0,WEST,Mountain South,NM,Mixed-Dry,144647.71,2656.89
1,SOUTH,West South Central,AR,Mixed-Humid,28034.61,975.00
2,WEST,Mountain South,NM,Mixed-Dry,30749.71,522.65
3,SOUTH,South Atlantic,SC,Mixed-Humid,86765.19,2061.77
4,NORTHEAST,Middle Atlantic,NJ,Mixed-Humid,59126.93,1463.04


In [4]:
#Checking nulls
regional_df.isnull().sum()

REGIONC         0
DIVISION        0
state_postal    0
BA_climate      0
TOTALBTU        0
TOTALDOL        0
dtype: int64

In [5]:
#Separate the features `X` from the target `y`
y = regional_df[['TOTALBTU', 'TOTALDOL']]
X = regional_df.drop(columns=['TOTALBTU', 'TOTALDOL'])

In [10]:
# Encode the categorical variables using get_dummies
X = pd.get_dummies(X)

# Review the features data
X.head()

,REGIONC_MIDWEST,REGIONC_NORTHEAST,REGIONC_SOUTH,REGIONC_WEST,DIVISION_East North Central,DIVISION_East South Central,DIVISION_Middle Atlantic,DIVISION_Mountain North,DIVISION_Mountain South,DIVISION_New England,...,state_postal_WV,state_postal_WY,BA_climate_Cold,BA_climate_Hot-Dry,BA_climate_Hot-Humid,BA_climate_Marine,BA_climate_Mixed-Dry,BA_climate_Mixed-Humid,BA_climate_Subarctic,BA_climate_Very-Cold
0,False,False,False,True,False,False,False,False,True,False,...,False,False,False,False,False,False,True,False,False,False
1,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,False,False,False,True,False,False,False,False,True,False,...,False,False,False,False,False,False,True,False,False,False
3,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,False,True,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [11]:
# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
# Define the model - deep neural net
number_input_features = X_train_scaled.shape[1]
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

In [14]:
# Initialize Sequential model
nn_model = tf.keras.Sequential()

# Add first hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Add output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="linear"))  # Assuming it's a regression task

# Print model summary
nn_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 592       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 643 (2.51 KB)
Trainable params: 643 (2.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, Y_train, epochs=10, verbose=1)

Epoch 1/10


463/463 [==============================] - 1s 853us/step - loss: -456883.5000 - accuracy: 0.0000e+00
Epoch 2/10
463/463 [==============================] - 0s 881us/step - loss: -647655.3125 - accuracy: 0.0000e+00
Epoch 3/10
463/463 [==============================] - 0s 844us/step - loss: -649427.1250 - accuracy: 0.0000e+00
Epoch 4/10
463/463 [==============================] - 0s 835us/step - loss: -649558.3750 - accuracy: 0.0000e+00
Epoch 5/10
463/463 [==============================] - 0s 897us/step - loss: -649558.5625 - accuracy: 0.0000e+00
Epoch 6/10
463/463 [==============================] - 0s 844us/step - loss: -649559.0625 - accuracy: 0.0000e+00
Epoch 7/10
463/463 [==============================] - 0s 838us/step - loss: -649559.3125 - accuracy: 0.0000e+00
Epoch 8/10
463/463 [==============================] - 0s 834us/step - loss: -649560.1250 - accuracy: 0.0000e+00
Epoch 9/10
463/463 [==============================] - 0s 848us/step - loss: -649563.7500 - accuracy: 0

In [20]:
# Evaluate the model using the test data
model_loss = nn_model.evaluate(X_test_scaled, Y_test, verbose=2)
print(f"Loss: {model_loss}")

116/116 - 0s - loss: -6.4175e+05 - accuracy: 0.0000e+00 - 85ms/epoch - 730us/step
Loss: [-641745.3125, 0.0]
